In [1]:
import requests
import spacy
import re
import spacy_transformers
import torch
from spacytextblob.spacytextblob import SpacyTextBlob
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

In [2]:
nlp = spacy.load('de_dep_news_trf')
nlp.add_pipe('spacytextblob')

/home/somber/anaconda3/lib/python3.9/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'de_dep_news_trf' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [21]:
response = requests.get("https://search.dip.bundestag.de/api/v1/drucksache-text?f.dokumentnummer=20/3450&apikey=GmEPb1B.bfqJLIhcGAsH9fTJevTglhFpCoZyAAAdhp")
print(response)

<Response [200]>


In [22]:
response.xml()

AttributeError: 'Response' object has no attribute 'xml'

In [9]:
data = response.json()

In [10]:
#print(data['documents'][0]['text'])

In [11]:
doc = nlp(data['documents'][0]['text'])

In [12]:
doc._.polarity

0.09999999999999999

In [13]:
for span in doc.sents:
    print(span.text, span._.polarity)
    print('')

Kleine Anfrage
der Fraktion der CDU/CSU
 0.0

Ergebnisse des zweiten Stresstests 0.0


 0.0

Die vier deutschen Übertragungsnetzbetreiber 50Hertz, Amprion, TenneT und 
Transnet haben am 5. September 2022 im Rahmen einer Pressekonferenz die 
Ergebnisse des zweiten Netzstresstests (zweite Sonderanalyse Winter 
2022/2023) vorgestellt. 0.0

Der Bundesminister für Wirtschaft und Klimaschutz 
Dr. Robert Habeck hat im Anschluss an die Vorstellung der Ergebnisse seine 
Schlüsse aus dem Stresstest dargelegt (https://www.bmwk.de/Redaktion/DE/Pr
essemitteilungen/2022/09/20220905-stresstest-zum-stromsystem.html). 0.0

Danach 
sollen „die beiden AKW Isar 2 und Neckarwestheim bis Mitte April 2023 noch 
zur Verfügung stehen, um, falls nötig, über den Winter einen zusätzlichen Bei-
trag im Stromnetz in Süddeutschland 2022/23 leisten zu können. 0.0

“ Eine einheit-
liche Auffassung der Bundesregierung zu diesen Erläuterungen des Bundes-
wirtschaftsministers scheint es bisher nicht zu geben (https://www

In [14]:
for span in doc.sents:
    blob = TextBlob(span.text, analyzer=NaiveBayesAnalyzer())
    print(span, blob.sentiment)
    print('')

Kleine Anfrage
der Fraktion der CDU/CSU
 Sentiment(classification='neg', p_pos=0.38888888888888884, p_neg=0.6111111111111108)

Ergebnisse des zweiten Stresstests Sentiment(classification='pos', p_pos=0.75, p_neg=0.2499999999999997)


 Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)

Die vier deutschen Übertragungsnetzbetreiber 50Hertz, Amprion, TenneT und 
Transnet haben am 5. September 2022 im Rahmen einer Pressekonferenz die 
Ergebnisse des zweiten Netzstresstests (zweite Sonderanalyse Winter 
2022/2023) vorgestellt. Sentiment(classification='pos', p_pos=0.9605433376455376, p_neg=0.039456662354463025)

Der Bundesminister für Wirtschaft und Klimaschutz 
Dr. Robert Habeck hat im Anschluss an die Vorstellung der Ergebnisse seine 
Schlüsse aus dem Stresstest dargelegt (https://www.bmwk.de/Redaktion/DE/Pr
essemitteilungen/2022/09/20220905-stresstest-zum-stromsystem.html). Sentiment(classification='neg', p_pos=0.333795367688436, p_neg=0.6662046323115643)



KeyboardInterrupt: 

In [17]:
#!pip install TextBlobDE

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ERROR: Could not find a version that satisfies the requirement TextBlobDE (from versions: none)
ERROR: No matching distribution found for TextBlobDE


In [15]:
from textblob_de import TextBlobDE
for span in doc.sents:
    blob = TextBlobDE(span.text)
    #print(blob.tags)
    #print(blob.noun_phrases)
    print(span, blob.sentiment)
    #print('')
    #for sentence in blob.sentences:
    #    print(sentence.sentiment.polarity)

Kleine Anfrage
der Fraktion der CDU/CSU
 Sentiment(polarity=0.0, subjectivity=0.0)
Ergebnisse des zweiten Stresstests Sentiment(polarity=0.0, subjectivity=0.0)

 Sentiment(polarity=0.0, subjectivity=0.0)
Die vier deutschen Übertragungsnetzbetreiber 50Hertz, Amprion, TenneT und 
Transnet haben am 5. September 2022 im Rahmen einer Pressekonferenz die 
Ergebnisse des zweiten Netzstresstests (zweite Sonderanalyse Winter 
2022/2023) vorgestellt. Sentiment(polarity=0.0, subjectivity=0.0)
Der Bundesminister für Wirtschaft und Klimaschutz 
Dr. Robert Habeck hat im Anschluss an die Vorstellung der Ergebnisse seine 
Schlüsse aus dem Stresstest dargelegt (https://www.bmwk.de/Redaktion/DE/Pr
essemitteilungen/2022/09/20220905-stresstest-zum-stromsystem.html). Sentiment(polarity=0.0, subjectivity=0.0)
Danach 
sollen „die beiden AKW Isar 2 und Neckarwestheim bis Mitte April 2023 noch 
zur Verfügung stehen, um, falls nötig, über den Winter einen zusätzlichen Bei-
trag im Stromnetz in Süddeutschland 20

Wenn die Frage 38 bejaht wurde, wie unterscheiden sich die Kosten für 
das Vorhalten der „Einsatzreserve“ von den laufenden Kosten im normalen 
Betrieb der Kernkraftwerke? Sentiment(polarity=0.7, subjectivity=0.0)

 Sentiment(polarity=0.0, subjectivity=0.0)
40. Sentiment(polarity=0.0, subjectivity=0.0)
Haben die Betreiber der Kernkraftwerke Isar 2 und Neckarwestheim der 
Bundesregierung den Betrieb im Rahmen der „Einsatzreserve“ zugesagt? Sentiment(polarity=1.0, subjectivity=0.0)

 Sentiment(polarity=0.0, subjectivity=0.0)
41. Sentiment(polarity=0.0, subjectivity=0.0)
Welche Gespräche mit den Betreibern der betreffenden Kernkraftwerke 
wurden zur Frage der Kosten ggf. bereits geführt? Sentiment(polarity=0.0, subjectivity=0.0)

 Sentiment(polarity=0.0, subjectivity=0.0)
42. Sentiment(polarity=0.0, subjectivity=0.0)
Hat die Bundesregierung bereits einen Regelungsvorschlag für die geplan-
te Erstattung der Kosten für die „Einsatzreserve“ an die Betreiber (https://
www.bmwk.de/Redaktion/DE

In [16]:
#!pip install spacy-sentiws

In [17]:
#!pip install spacy-sentiws
import spacy
from spacy_sentiws import spaCySentiWS

nlp = spacy.load('de_core_news_sm')
nlp.add_pipe('sentiws', config={'sentiws_path': 'data/sentiws'})
doc = nlp('Die Dummheit der Unterwerfung blüht in hübschen Farben.')

for token in doc:
    print('{}, {}, {}'.format(token.text, token._.sentiws, token.pos_))

/home/somber/anaconda3/lib/python3.9/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'de_core_news_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Die, None, DET
Dummheit, -0.4877, NOUN
der, None, DET
Unterwerfung, -0.3279, NOUN
blüht, 0.2028, VERB
in, None, ADP
hübschen, 0.4629, ADJ
Farben, None, NOUN
., None, PUNCT


In [18]:
from germansentiment import SentimentModel

model = SentimentModel()

texts = [
    "Mit keinem guten Ergebniss","Das ist gar nicht mal so gut",
    "Total awesome!","nicht so schlecht wie erwartet",
    "Der Test verlief positiv.","Sie fährt ein grünes Auto."]
       
result = model.predict_sentiment(texts)
print(result)

['negative', 'negative', 'positive', 'positive', 'neutral', 'neutral']


In [48]:
from germansentiment import SentimentModel
model = SentimentModel()

for span in doc.sents:
    print(span.text)
    print('')
    result = model.predict_sentiment(str(span.text))
    print(result)

Kleine Anfrage
der Fraktion der CDU/CSU


['negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative']
Ergebnisse des zweiten Stresstests

['negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative']



['negative']
Die vier deutsch

['negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'po

['negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'neg

['negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'ne

['negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'neutral', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'pos

...

['negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'ne

['negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'po

['positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'neutral', 'negative', 'positive', 'positive', 'positive', 'neutral', 'negative', 'positive', 'positive', 'negative']
Wie unterscheidet sich die notwendige Personalbesetzung nach Kenntnis 
der Bundesregierung im Rahmen der „Einsatzreserve“ von der Personalbe-
setzung im üblichen Kernkraftwerksbetrieb?
46.

['positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negat

['positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'po

['negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'neg

In [19]:
import json
import spacy
from textblob_de import TextBlobDE
nlp = spacy.load('de_core_news_sm')

with open('BundestagsRedenSample.json') as json_file:
    data = json.load(json_file)

kommentarlisteMitSentiment = {}
sentimentwert = 0
for rede in data:
    for kommentare in data[rede][2]:
        doc = nlp(kommentare[1])
        #doc._.polarity
        for span in doc.sents:
            # cia 5 Sekunden pro Kommentar
            blob = TextBlobDE(span.text)
            #print(span, blob.sentiment.polarity)
            sentimentwert = blob.sentiment.polarity
            #print('')
        kommentare.append(sentimentwert)
        #print(kommentare[1])
        
kommentarlisteMitSentiment = data

for rede in kommentarlisteMitSentiment:
    for kommentare in data[rede][2]:
        print(rede, '->', kommentare)

ID20100200 -> ['[AfD]', 'aber die geschäftsordnung!', 0.0]
ID20100200 -> ['[AfD]', 'dafür haben sie auch die quittung gekriegt!', 0.0]
ID20100200 -> ['[DIE LINKE]', 'das war ganz gut!', 1.0]
ID20100200 -> ['[AfD]', 'na ja!', 0.0]
ID20100300 -> ['[AfD]', 'das erkläre ich ihnen gleich!', 0.0]
ID20100300 -> ['[AfD]', 'sie haben den antrag nicht verstanden!', 0.0]
ID20100300 -> ['[AfD]', 'das ist ihre lebenswirklichkeit!', 0.0]
ID20100400 -> ['[DIE LINKE]', 'oder jugendweihe!', 0.0]
ID20100500 -> ['[BÜNDNIS 90/DIE GRÜNEN]', 'sie wollen keine afd!', 0.0]
ID20100500 -> ['[BÜNDNIS 90/DIE GRÜNEN]', 'ja!', 0.0]
ID20100500 -> ['[DIE LINKE]', 'ja, das hat sie doch gesagt!', 0.0]
ID20100500 -> ['[BÜNDNIS 90/DIE GRÜNEN]', 'o mein gott!', 0.0]
ID20100500 -> ['[CDU/CSU]', 'das ist falsch!', -1.0]
ID20100500 -> ['[CDU/CSU]', 'ja, in einem bundesland! in einem einzigen bundesland! der vollhonk! unmöglich! unfassbar!', 0.0]
ID20100600 -> ['[AfD]', 'jetzt kommt die rote widerlichkeit! das ist noch schlim

In [20]:
from spacy_sentiws import spaCySentiWS
nlp.add_pipe('sentiws', config={'sentiws_path': 'data/sentiws'})

with open('BundestagsRedenSample.json') as json_file:
    data = json.load(json_file)

kommentarlisteMitSentiment = {}
sentimentwert = 0
for rede in data:
    for kommentare in data[rede][2]:
        doc = nlp(kommentare[1])
        #doc._.polarity
        for token in doc:
            print('{}, {}'.format(token.text, token._.sentiws))
        #kommentare.append(sentimentwert)
        #print(kommentare[1])
        
kommentarlisteMitSentiment = data

#for rede in kommentarlisteMitSentiment:
#    for kommentare in data[rede][2]:
#        print(rede, '->', kommentare)

aber, None
die, None
geschäftsordnung, None
!, None
dafür, None
haben, None
sie, None
auch, None
die, None
quittung, None
gekriegt, None
!, None
das, None
war, None
ganz, None
gut, None
!, None
na, None
ja, None
!, None
das, None
erkläre, None
ich, None
ihnen, None
gleich, None
!, None
sie, None
haben, None
den, None
antrag, None
nicht, None
verstanden, 0.0957
!, None
das, None
ist, None
ihre, None
lebenswirklichkeit, None
!, None
oder, None
jugendweihe, None
!, None
sie, None
wollen, None
keine, None
afd, None
!, None
ja, None
!, None
ja, None
,, None
das, None
hat, None
sie, None
doch, None
gesagt, None
!, None
o, None
mein, None
gott, None
!, None
das, None
ist, None
falsch, None
!, None
ja, None
,, None
in, None
einem, None
bundesland, None
!, None
in, None
einem, None
einzigen, None
bundesland, None
!, None
der, None
vollhonk, None
!, None
unmöglich, None
!, None
unfassbar, None
!, None
jetzt, None
kommt, None
die, None
rote, None
widerlichkeit, None
!, None
das, None
ist, None
no